# Movie Project: Extracting Data
- Zach Hanson

## Libraries and Functions

### Libraries

In [1]:
#Standard imports
import pandas as pd
import numpy as np

#JSON stuff
import os
import time
import json

#TMDB
import tmdbsimple as tmdb

#tqdm progress bar
from tqdm.notebook import tqdm_notebook

### Functions

#### Write to JSON Folder

In [2]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

#### Getting the Certification of the Given Movie

In [3]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    #Loop through the countries in releases
    for c in releases['countries']:
        #if the country abbreviation is 'US'
        if c['iso_3166_1'] == 'US':
            #Save a "certification" key in info with the certification
            info['certification'] = c['certification']
            
    return info

## Loading in Data

### Loading login credentials for tmdb api

In [4]:
with open('/Users/Zach/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)

#Keys in dictionary
login.keys()

dict_keys(['api-key'])

In [5]:
#Setting api key
tmdb.API_KEY = login['api-key']

### Loading Data

In [6]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_results_combined.csv.gz']

In [7]:
#Loading in dataframe from "Filter.ipynb"
basics = pd.read_csv('Data/title_basics.csv.gz')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
85562,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
85563,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
85564,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
85565,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


- Loaded in properly

In [8]:
#What years we want to extract, for now just 2000 and 2001
YEARS = [2000, 2001]

In [9]:
#Creating error list to capture any errors in our loop
errors = []

In [21]:
#START OUTER LOOP
for YEAR in tqdm_notebook(YEARS, desc = 'YEARS', position = 0):
    
    #Defining JSON to store results
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #Check if the file already exists
    file_exists = os.path.isfile(JSON_FILE)
    #If file does not exist
    if file_exists == False:
        #Create it
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}],f)
            
    #Define and filter the movie ID's to call in inner loop
    df = basics.loc[basics['startYear'] == YEAR].copy()
    #Save movie ID to list
    movie_ids = df['tconst'].copy()
    
    #Load existing data into a dataframe
    previous_df = pd.read_json(JSON_FILE)
    #Filter out movies that are already in the file to prevent duplicating
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    
    #START INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc = f'Movies from {YEAR}',
                                  position = 1,
                                  leave = True):
        try:
            #Retriece data for the movie id
            temp = get_movie_with_rating(movie_id)
            #Append results to file
            write_json(temp, JSON_FILE)
            #Delay to not overload server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    #Saving JSON file as a smaller file format
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                         compression = 'gzip',
                         index = False)
    
#Number of movies that caused an error
print(f'Total Errors: {len(errors)}')

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1432 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1548 [00:00<?, ?it/s]

Total Errors: 3435


## Loading the Rest of the Data for Testing

In [11]:
#What years we want to extract, 2002-2019
"""
Since the COVID pandemic occurred at the start of 2020, most likely shutting down
the vast majority of movie theatres and as a result reducing the potential profit 
of most movies. Because of this, we will only look up to 2019 to do our testing.
""" 

YEARS = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
         2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
YEARS

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [12]:
#START OUTER LOOP
for YEAR in tqdm_notebook(YEARS, desc = 'YEARS', position = 0):
    
    #Defining JSON to store results
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #Check if the file already exists
    file_exists = os.path.isfile(JSON_FILE)
    #If file does not exist
    if file_exists == False:
        #Create it
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}],f)
            
    #Define and filter the movie ID's to call in inner loop
    df = basics.loc[basics['startYear'] == YEAR].copy()
    #Save movie ID to list
    movie_ids = df['tconst'].copy()
    
    #Load existing data into a dataframe
    previous_df = pd.read_json(JSON_FILE)
    #Filter out movies that are already in the file to prevent duplicating
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    
    #START INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc = f'Movies from {YEAR}',
                                  position = 1,
                                  leave = True):
        try:
            #Retriece data for the movie id
            temp = get_movie_with_rating(movie_id)
            #Append results to file
            write_json(temp, JSON_FILE)
            #Delay to not overload server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    #Saving JSON file as a smaller file format
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                         compression = 'gzip',
                         index = False)
    
#Number of movies that caused an error
print(f'Total Errors: {len(errors)}')

YEARS:   0%|          | 0/18 [00:00<?, ?it/s]

Movies from 2002:   0%|          | 0/1549 [00:00<?, ?it/s]

Movies from 2003:   0%|          | 0/1666 [00:00<?, ?it/s]

Movies from 2004:   0%|          | 0/1882 [00:00<?, ?it/s]

Movies from 2005:   0%|          | 0/2167 [00:00<?, ?it/s]

...

Movies from 2016:   0%|          | 0/5218 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5589 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5713 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5819 [00:00<?, ?it/s]

Total Errors: 16213
